In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# upload train dataset
from google.colab import files

# Open file picker
uploaded = files.upload()
train_df = pd.read_csv("train_v9rqX0R.csv")

Saving train_v9rqX0R.csv to train_v9rqX0R.csv


In [ ]:
# upload test dataset
from google.colab import files

# Open file picker
uploaded = files.upload()
valid_df = pd.read_csv("test_AbJTz2l.csv")

Saving test_AbJTz2l.csv to test_AbJTz2l.csv


In [ ]:
# --- Load data in jupyter notebook ---
# train_df = pd.read_csv("train.csv")
# valid_df = pd.read_csv("test.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
# check null values
train_df.isnull().sum(axis=0)

In [ ]:
# 1) Items in Column 'Item_Fat_Content' has variable naming. Standardize Item_Fat_Content
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].replace({
    'low fat': 'Low Fat',
    'LF': 'Low Fat',
    'reg': 'Regular'
})
valid_df['Item_Fat_Content'] = valid_df['Item_Fat_Content'].replace({
    'low fat': 'Low Fat',
    'LF': 'Low Fat',
    'reg': 'Regular'
})

In [ ]:
train_df.head()

In [ ]:
# # 1) Handle missing values in column "Outlet_Size"
# Outlet size may depend on outlet stablishment year, outlet type and output location type. So, skipping these rows or just filling "Small" may not bbe efficient
# so matrix comparison using cross table would be useful to undestand their relationship

crosstable = pd.crosstab(
    index=df['Outlet_Size'],
    columns=[df['Outlet_Type'], df['Outlet_Establishment_Year']]
)
crosstable

In [ ]:
# 2) Handle missing values in Outlet_Size. Grocery Store is categorized as Small. Supermarket Type2 and Supermarket Type3 are Medium. Supermarket Type1 has all three categories (so do not consider)
size_map = {
    'Grocery Store': 'Small',
    'Supermarket Type2': 'Medium',
    'Supermarket Type3': 'Medium'
}
train_df['Outlet_Size'] = train_df['Outlet_Size'].fillna(train_df['Outlet_Type'].map(size_map))
valid_df['Outlet_Size'] = valid_df['Outlet_Size'].fillna(valid_df['Outlet_Type'].map(size_map))

In [ ]:
train_df.head()

In [ ]:
# Still some values of Outlet_Size are zero

In [ ]:
crosstable = pd.crosstab(
    index=df['Outlet_Size'],
    columns=df['Outlet_Location_Type']
)
crosstable

In [ ]:
# 2.1) Handle missing values based on Outlet_Location_Type. As Tier 2 is categorized under Small in crosstable
size_map2 = {'Tier 2': 'Small'}
train_df['Outlet_Size'] = train_df['Outlet_Size'].fillna(train_df['Outlet_Location_Type'].map(size_map2))
valid_df['Outlet_Size'] = valid_df['Outlet_Size'].fillna(valid_df['Outlet_Location_Type'].map(size_map2))

In [ ]:
train_df.head()

In [ ]:
# 3) Handle missing values in Item_Weight. Group Item_Weight based on Item_Identifier and fill the mean value
train_df['Item_Weight'] = train_df['Item_Weight'].fillna(train_df.groupby(['Item_Identifier'])['Item_Weight'].transform('mean'))
valid_df['Item_Weight'] = valid_df['Item_Weight'].fillna(valid_df.groupby(['Item_Identifier'])['Item_Weight'].transform('mean'))

In [ ]:
train_df.isnull(sum=0)

In [ ]:
# Still there are missing values in Item_Weight. Group them based on Item_Type and fill the mean value
train_df['Item_Weight'] = train_df['Item_Weight'].fillna(train_df.groupby('Item_Type')['Item_Weight'].transform('mean'))
valid_df['Item_Weight'] = valid_df['Item_Weight'].fillna(valid_df.groupby('Item_Type')['Item_Weight'].transform('mean'))

In [ ]:
# 4) Handle zeros in Item_Visibility. Group by Item_Identifier and fill in the mean values
train_df['Item_Visibility'] = train_df.groupby('Item_Identifier')['Item_Visibility'].transform(
    lambda x: x.replace(0, x.mean())
)
valid_df['Item_Visibility'] = valid_df.groupby('Item_Identifier')['Item_Visibility'].transform(
    lambda x: x.replace(0, x.mean())
)

# Some values are still zero in validation set. As they don't multiple products (repeated Item_Identifier). Fill remaining zeros in test
valid_df['Item_Visibility'] = valid_df.groupby(['Item_Type', 'Item_Fat_Content'])['Item_Visibility'].transform('mean')

In [ ]:
# --- Calculate Outlet_Age. As it has an effect on sales
train_df['Outlet_Age'] = 2025 - train_df['Outlet_Establishment_Year']
valid_df['Outlet_Age'] = 2025 - valid_df['Outlet_Establishment_Year']

In [ ]:
# --- Encoding string columns
# Item_Fat_Content → 1,2
train_df['Item_Fat_Content'] = train_df['Item_Fat_Content'].replace({'Low Fat': 1, 'Regular': 2})
valid_df['Item_Fat_Content'] = valid_df['Item_Fat_Content'].replace({'Low Fat': 1, 'Regular': 2})

# Outlet_Size → 1,2,3
train_df['Outlet_Size'] = train_df['Outlet_Size'].replace({'Small': 1, 'Medium': 2, 'High': 3})
valid_df['Outlet_Size'] = valid_df['Outlet_Size'].replace({'Small': 1, 'Medium': 2, 'High': 3})

# Outlet_Location_Type → 1,2,3
train_df['Outlet_Location_Type'] = train_df['Outlet_Location_Type'].replace({'Tier 1': 1, 'Tier 2': 2, 'Tier 3': 3})
valid_df['Outlet_Location_Type'] = valid_df['Outlet_Location_Type'].replace({'Tier 1': 1, 'Tier 2': 2, 'Tier 3': 3})

# Outlet_Type → 1,2,3,4
outlet_type_map = {
    'Grocery Store': 1,
    'Supermarket Type1': 2,
    'Supermarket Type2': 3,
    'Supermarket Type3': 4
}
train_df['Outlet_Type'] = train_df['Outlet_Type'].replace(outlet_type_map)
valid_df['Outlet_Type'] = valid_df['Outlet_Type'].replace(outlet_type_map)

# --- Encode Item_Type as 1,2,3,... ---
train_df['Item_Type'], uniques = pd.factorize(train_df['Item_Type'])
valid_df['Item_Type'] = pd.Categorical(valid_df['Item_Type'], categories=uniques).codes

train_df['Item_Type'] = train_df['Item_Type'] + 1
valid_df['Item_Type'] = valid_df['Item_Type'] + 1

In [ ]:
# --- Without Feature Engineering, XGBoost RMSE obtained a saturation of 1089, Linear Regression RMSE 1212, Polynomial Regression RMSE 1090, Randon Forest RMSE 1132

# --- Feature Engineering 1 : Item_Sales_Frequency ---
# ---------- Trial 1 - xgboost rmse - 1041 (Lowest rmse mentioned)
#train_df['Item_Sales_Frequency'] = train_df['Outlet_Age'] * (train_df['Item_MRP'] - train_df['Item_Visibility'])/(train_df['Item_Weight'] + 1)
#valid_df['Item_Sales_Frequency'] = valid_df['Outlet_Age'] * (valid_df['Item_MRP'] - valid_df['Item_Visibility'])/(valid_df['Item_Weight'] + 1)

#item_popularity = train_df['Item_Identifier'].value_counts(normalize=True)  # normalized frequency
#train_df['Item_Popularity'] = train_df['Item_Identifier'].map(item_popularity)
#valid_df['Item_Popularity'] = valid_df['Item_Identifier'].map(item_popularity).fillna(0)  # unseen items → 0

# ---------- Trial 2 - polynomial regression rmse - 1039 (Lowest rmse mentioned)
#train_df['Item_Sales_Frequency'] = (
#    np.log1p(train_df['Outlet_Age']) * (train_df['Item_MRP'] / (train_df['Item_Weight'] + 1)) * train_df['Item_Popularity']
#)

#valid_df['Item_Sales_Frequency'] = (
#    np.log1p(valid_df['Outlet_Age']) * (valid_df['Item_MRP'] / (valid_df['Item_Weight'] + 1)) * valid_df['Item_Popularity']
#)

# ---------- Trial 3 - polynomial regression - 1038.26 (Lowest rmse mentioned)
item_popularity = train_df['Item_Identifier'].value_counts(normalize=True)  # normalized frequency

train_df['Item_Popularity'] = train_df['Item_Identifier'].map(item_popularity)
valid_df['Item_Popularity'] = valid_df['Item_Identifier'].map(item_popularity).fillna(0)  # unseen items → 0

train_df['Item_Sales_Frequency'] = (
    np.log1p(train_df['Outlet_Age']) *
    ((train_df['Item_MRP'] - train_df['Item_MRP'].mean()) / (train_df['Item_MRP'].std() + 1)) *
    (train_df['Item_Popularity'] + 0.01)  # smoothing
)

valid_df['Item_Sales_Frequency'] = (
    np.log1p(valid_df['Outlet_Age']) *
    ((valid_df['Item_MRP'] - valid_df['Item_MRP'].mean()) / (valid_df['Item_MRP'].std() + 1)) *
    (valid_df['Item_Popularity'] + 0.01)
)


# --- Handle Inf / NaN in Item_Sales_Frequency ---
train_df['Item_Sales_Frequency'].replace([np.inf, -np.inf], np.nan, inplace=True)
valid_df['Item_Sales_Frequency'].replace([np.inf, -np.inf], np.nan, inplace=True)
train_df['Item_Sales_Frequency'].fillna(train_df['Item_Sales_Frequency'].mean(), inplace=True)
valid_df['Item_Sales_Frequency'].fillna(valid_df['Item_Sales_Frequency'].mean(), inplace=True)

In [ ]:
# --- Feature Engineering 2 : Customer Outlet Preference ---
outlet_type_sales = train_df.groupby('Outlet_Type')['Item_Outlet_Sales'].sum()
outlet_type_percentage = outlet_type_sales / outlet_type_sales.sum()

train_df['Outlet_Type_Percentage'] = train_df['Outlet_Type'].map(outlet_type_percentage)
valid_df['Outlet_Type_Percentage'] = valid_df['Outlet_Type'].map(outlet_type_percentage)

# ---------- Trial 3 - xgboost rmse - 1041
#train_df['Customer_Outlet_Preference'] = (
#    train_df['Item_MRP'] * train_df['Outlet_Type_Percentage'] / (train_df['Item_Weight']+1)*(train_df['Item_Visibility']+1)
#)
#valid_df['Customer_Outlet_Preference'] = (
#    valid_df['Item_MRP'] * valid_df['Outlet_Type_Percentage'] / (valid_df['Item_Weight']+1)*(valid_df['Item_Visibility']+1)
#)

# polynomial regression - 1039
#median_mrp = df['Item_MRP'].median()

#train_df['Customer_Outlet_Preference'] = (
#    ((train_df['Item_MRP'] / median_mrp) ** 0.5) *
#    np.exp(-train_df['Item_Visibility']) *
#    train_df['Outlet_Type_Percentage']
#)

#valid_df['Customer_Outlet_Preference'] = (
#    ((valid_df['Item_MRP'] / median_mrp) ** 0.5) *
#    np.exp(-valid_df['Item_Visibility']) *
#    valid_df['Outlet_Type_Percentage']
#)


# ---------- Trial 2 - polynomial regression - 1038.26 (lowest rmse mentioned)
median_mrp = train_df['Item_MRP'].median()

train_df['Customer_Outlet_Preference'] = (
    np.sqrt(train_df['Item_MRP'] / median_mrp) *
    (1 / (1 + np.log1p(train_df['Item_Visibility']))) *   # softer visibility penalty
    (train_df['Outlet_Type_Percentage'] * (1 / train_df['Outlet_Location_Type']))
)

valid_df['Customer_Outlet_Preference'] = (
    np.sqrt(valid_df['Item_MRP'] / median_mrp) *
    (1 / (1 + np.log1p(valid_df['Item_Visibility']))) *
    (valid_df['Outlet_Type_Percentage'] * (1 / valid_df['Outlet_Location_Type']))
)

In [ ]:
# Feature Engieering 3
# polynomial regression + Feature Engieering 1 + Feature Engieering 2 + Feature Engieering 3 - RMSE = 1038.14

# 1. Price per weight
train_df['Price_per_Weight'] = train_df['Item_MRP'] / (train_df['Item_Weight'] + 1)
valid_df['Price_per_Weight'] = valid_df['Item_MRP'] / (valid_df['Item_Weight'] + 1)

# 2. Visibility × Price interaction
train_df['Visibility_Price_Interaction'] = train_df['Item_Visibility'] * train_df['Item_MRP']
valid_df['Visibility_Price_Interaction'] = valid_df['Item_Visibility'] * valid_df['Item_MRP']

# 3. Outlet Age × Outlet Type
train_df['Outlet_Age_Type'] = train_df['Outlet_Age'] * train_df['Outlet_Type']
valid_df['Outlet_Age_Type'] = valid_df['Outlet_Age'] * valid_df['Outlet_Type']

# 4. Fat × Type interaction
train_df['Fat_Type_Interaction'] = train_df['Item_Fat_Content'] * train_df['Item_Type']
valid_df['Fat_Type_Interaction'] = valid_df['Item_Fat_Content'] * valid_df['Item_Type']

# 5. Outlet diversity
outlet_diversity = train_df.groupby('Outlet_Identifier')['Item_Identifier'].nunique()
train_df['Outlet_Diversity'] = train_df['Outlet_Identifier'].map(outlet_diversity)
valid_df['Outlet_Diversity'] = valid_df['Outlet_Identifier'].map(outlet_diversity)
valid_df['Outlet_Diversity'].fillna(outlet_diversity.mean(), inplace=True)

# 6. Log features
train_df['Log_MRP'] = np.log1p(train_df['Item_MRP'])
valid_df['Log_MRP'] = np.log1p(valid_df['Item_MRP'])

train_df['Log_Visibility'] = np.log1p(train_df['Item_Visibility'])
valid_df['Log_Visibility'] = np.log1p(valid_df['Item_Visibility'])

In [ ]:
# Normalize to 0-1 range (avoid gradient explosion)
#train_df['Customer_Outlet_Preference'] = (
#    (train_df['Customer_Outlet_Preference'] - train_df['Customer_Outlet_Preference'].min()) /
#    (train_df['Customer_Outlet_Preference'].max() - train_df['Customer_Outlet_Preference'].min())
#)
#valid_df['Customer_Outlet_Preference'] = (
#    (valid_df['Customer_Outlet_Preference'] - valid_df['Customer_Outlet_Preference'].min()) /
#    (valid_df['Customer_Outlet_Preference'].max() - valid_df['Customer_Outlet_Preference'].min())
#)

In [ ]:
features = [
    'Item_Weight', 'Item_Fat_Content', 'Item_Type', 'Item_MRP',
    'Item_Visibility', 'Outlet_Size', 'Outlet_Location_Type',
    'Outlet_Type', 'Outlet_Age',
    'Item_Sales_Frequency', 'Customer_Outlet_Preference',
    'Price_per_Weight', 'Visibility_Price_Interaction',
    'Outlet_Age_Type', 'Fat_Type_Interaction',
    'Outlet_Diversity', 'Log_MRP', 'Log_Visibility'
]


X = train_df[features]
y = train_df['Item_Outlet_Sales']
X_test_final = valid_df[features]

# --- Train/Validation Split ---
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)